In [1]:
# Import packages
import pandas as pd
import numpy as np
import os
import statsmodels.api as sm
from statsmodels.formula.api import ols
from statsmodels.formula.api import logit
import matplotlib.pyplot as plt
import json
import seaborn as sns
from IPython.display import HTML

In [2]:
answer_key = {"26": 1, "27": 1, "28": 2, "29": 1, "30": 2, "31": 1, "32": 2, "33": 2, "34": 2, "35": 2, "36": 1, "37": 1, "38": 2, "39": 2, "40": 1, "41": 2, "42": 1, "43": 2, "44": 2, "45": 2, "46": 2, "47": 1, "48": 1, "49": 2, "50": 2}

In [3]:
def mapValues(df, orig_col, new_col, map_dict):
    df[new_col] = df[orig_col].map(map_dict)
    print("Assigned", new_col)
    return df
def multiValuePctTotal(df, custom_map, value_name):
    temp = {value_name: [], 'Percent of Total':[]}
    for key, value in custom_map.items():
        temp[value_name].append(value['dimension'])
        temp['Percent of Total'].append(value['count']/len(df))
    return pd.DataFrame(temp).set_index(value_name)#.sort_values(by='Percent of Total', ascending=False)
def clicked(arg):
    print("button has been clicked!")

In [4]:
df = pd.read_csv('adhd_v2.csv')
print("There are", df.shape[0], "responses in the original file. We will only be analyzing teachers who completed the survey.")
df = df[(df['Progress']==100) & (df['Q4'].notnull()) & (df['Q8'].notnull())].reset_index(drop=True)
print("Analyzing", df.shape[0], "responses.")
print()
print()
print("Cleaning up data...")
# Clean up vignettes to use gender
df["student_gender"] = np.where((df["FL_16_DO"] == "Block3") | (df["FL_16_DO"] == "Block6"), "male", "female")
df["adhd_subtype"] = np.where((df["FL_16_DO"] == "Block3") | (df["FL_16_DO"] == "Block7"), "inattentive", "combined")
df = mapValues(df, 'FL_16_DO','Vignette Type', {"Block3":'Inattentive Male', 'Block6':'Combined Male', 'Block5':'Combined Female', 'Block7':'Inattentive Female'})

print("Assigned Vignette attributes: Student Gender and ADHD Subtype")

# Clean up teaching years col
df.loc[df['Q8']=='16 years', 'Q8'] = 16
df['Q8'] = df['Q8'].astype('int')
df['years_of_teaching'] = np.where(df["Q8"] < 5, "<5 Years",
                                   np.where((df["Q8"]>=5) & (df["Q8"]<=10), "5-10 Years",
                                           np.where(df["Q8"]>10, ">10 Years","Other")))
print("Assigned Years of Teaching.")


# Clean up teacher gender
df["teacher_gender"] = np.where(df["Q4"] == 1, "male", "female")
print("Assigned Teacher Gender")

# Clean up additional certs
df['Additional Certification'] = np.where(df["Q9 - CERT"] == "Yes", "Yes", "No")
print("Assigned additional certification")

# Clean up encoded values
df = mapValues(df, 'Q11','School Setting', {1:'Public', 2:'Private', 3:'Charter',4:'Other'})
df = mapValues(df, 'Q12','Geographical Area', {1:'Urban', 2:'Suburban', 3:'Rural'})

grades_currently_teaching_map = {
                                1 : {'dimension':'Kindergarten', 'count': 0},
                                2 : {'dimension':'1st', 'count': 0},
                                3 : {'dimension':'2nd', 'count': 0},
                                4 : {'dimension':'3rd', 'count': 0},
                                5 : {'dimension':'4th', 'count': 0},
                                6 : {'dimension':'5th', 'count': 0},
                                7 : {'dimension':'6th', 'count': 0},
                                8 : {'dimension':'7th', 'count': 0},
                                9 : {'dimension':'Other', 'count': 0}
                            }
grades_taught_in_past_map = {
                            1 : {'dimension':'Kindergarten', 'count': 0},
                            2 : {'dimension':'1st', 'count': 0},
                            3 : {'dimension':'2nd', 'count': 0},
                            4 : {'dimension':'3rd', 'count': 0},
                            5 : {'dimension':'4th', 'count': 0},
                            6 : {'dimension':'5th', 'count': 0},
                            7 : {'dimension':'6th', 'count': 0},
                            8 : {'dimension':'7th', 'count': 0},
                            9 : {'dimension':'8th', 'count': 0},
                            10 : {'dimension':'9th', 'count': 0},
                            11 : {'dimension':'10th', 'count': 0},
                            12 : {'dimension':'11th', 'count': 0},
                            13 : {'dimension':'12th', 'count': 0},
                            14 : {'dimension':'Other', 'count': 0}
                            }
type_of_classroom_map = {
                        1 : {'dimension':'General Education', 'count': 0},
                        2 : {'dimension':'Special Education', 'count': 0},
                        3 : {'dimension':'Integrated/Co-taught', 'count': 0},
                        4 : {'dimension':'Other', 'count': 0}
                        }
teachers_view_map = {
                        1 : {'dimension':'1', 'count': 0},
                        2 : {'dimension':'2', 'count': 0},
                        3 : {'dimension':'3', 'count': 0},
                        4 : {'dimension':'4', 'count': 0},
                        5 : {'dimension':'ADHD', 'count': 0},
                        6 : {'dimension':'6', 'count': 0},
                        7 : {'dimension':'7', 'count': 0}
                    }
student_exhibits_adhd = []
for i, row in df.iterrows():
    for grade in str(row['Q13']).split(','):
        grades_currently_teaching_map[int(grade)]['count']+=1
    if row['Q14'] is not np.nan:
        for past_grade in str(row['Q14']).split(','):
            grades_taught_in_past_map[int(past_grade)]['count']+=1
    for type_room in str(row['Q15']).split(','):
        type_of_classroom_map[int(type_room)]['count']+=1
    for teacher_view in str(row['Q22']).split(','):
        teachers_view_map[int(teacher_view)]['count']+=1
    if '5' in str(row['Q22']):
        student_exhibits_adhd.append(1)
    else:
        student_exhibits_adhd.append(0)
df['student_exhibits_adhd'] = student_exhibits_adhd    
print("Generated counts for grades currently teaching, grades taught in the past, and type of classroom.")
# Rename columns
rename_dict = { 'Q22':'teachers_view', 'Q25_1':'degree_of_need'}
df = df.rename(columns=rename_dict)
df.head()

There are 50 responses in the original file. We will only be analyzing teachers who completed the survey.
Analyzing 50 responses.


Cleaning up data...
Assigned Vignette Type
Assigned Vignette attributes: Student Gender and ADHD Subtype
Assigned Years of Teaching.
Assigned Teacher Gender
Assigned additional certification
Assigned School Setting
Assigned Geographical Area
Generated counts for grades currently teaching, grades taught in the past, and type of classroom.


,Progress,Duration (in seconds),Finished,ResponseId,Q1,Q4,Q5,Q8,Q9,Q9 - CERT,...,FL_16_DO,student_gender,adhd_subtype,Vignette Type,years_of_teaching,teacher_gender,Additional Certification,School Setting,Geographical Area,student_exhibits_adhd
0,100,570,1,R_2XmMOwXkxl03RS1,1,2,26,5,Elementary and early childhood education (BS),No,...,Block3,male,inattentive,Inattentive Male,5-10 Years,female,No,Public,Suburban,0
1,100,319,1,R_323Zdlt2e8eOHAZ,1,2,25,4,Bachelors degree- early childhood education,No,...,Block3,male,inattentive,Inattentive Male,<5 Years,female,No,Public,Suburban,1
2,100,275,1,R_3KOu9RKNFFUTc9T,1,2,41,11,M Ed,No,...,Block7,female,inattentive,Inattentive Female,>10 Years,female,No,Private,Suburban,1
3,100,530,1,R_31j7NdjFkRxxvT5,1,2,27,6,Childhood Special Education,Yes,...,Block5,female,combined,Combined Female,5-10 Years,female,Yes,Private,Urban,1
4,100,346,1,R_2tEx8uK4n6xh7HF,1,2,23,2,NaN,NaN,...,Block5,female,combined,Combined Female,<5 Years,female,No,Private,Urban,1


In [5]:
print("Calculating teacher knowledge of ADHD...")
grades = []
for i, row in df.iterrows():
    curr_grade = 0
    for question_num in range(26,51):
        col = 'Q'+str(question_num)
        if row[col]==answer_key[str(question_num)]:
            curr_grade += 1
    grades.append(curr_grade)
    if curr_grade == 0:
        print(row)
df['teacher_knowledge'] = grades

Calculating teacher knowledge of ADHD...


# Descriptive Statistics

## 1. Demographic Tables (Percent of Total)

In [6]:
# % Total Descriptive Statistics
desc_columns=['teacher_gender', 'years_of_teaching', 'School Setting', 'Geographical Area', 'Additional Certification']
for col in desc_columns:
    display(pd.DataFrame(df[col].value_counts() / len(df)))

,teacher_gender
female,0.94
male,0.06


,years_of_teaching
>10 Years,0.50
5-10 Years,0.34
<5 Years,0.16


,School Setting
Public,0.72
Private,0.22
Charter,0.04
Other,0.02


,Geographical Area
Suburban,0.8
Urban,0.2


,Additional Certification
Yes,0.76
No,0.24


In [7]:
display(multiValuePctTotal(df, grades_currently_teaching_map, 'Grades Currently Teaching'))
display(multiValuePctTotal(df, grades_taught_in_past_map, 'Grades Taught in Past'))
display(multiValuePctTotal(df, type_of_classroom_map, 'Type of Classroom'))

,Percent of Total
Grades Currently Teaching,
Kindergarten,0.14
1st,0.18
2nd,0.22
3rd,0.32
4th,0.32
5th,0.22
6th,0.10
7th,0.14
Other,0.10


,Percent of Total
Grades Taught in Past,
Kindergarten,0.46
1st,0.44
2nd,0.48
3rd,0.50
4th,0.60
5th,0.42
6th,0.24
7th,0.16
8th,0.12


,Percent of Total
Type of Classroom,
General Education,0.40
Special Education,0.26
Integrated/Co-taught,0.34
Other,0.24


## 2. Descriptive Table - (% of participants presented with each vignette that selected ADHD on Q22)

In [8]:
# pd.DataFrame(
vign_correct_sum = pd.DataFrame(df.groupby('Vignette Type').sum(numeric_only=True)['student_exhibits_adhd'])
vign_cnt = pd.DataFrame(df.groupby('Vignette Type').count()['student_exhibits_adhd']).rename(columns = {'student_exhibits_adhd':'total'})
df_vign = pd.concat([vign_correct_sum,vign_cnt],axis=1)
df_vign['Percent of Participants That Selected ADHD by Vignette Type'] = df_vign['student_exhibits_adhd']/df_vign['total']
df_vign
# ) # / len(df)))

,student_exhibits_adhd,total,Percent of Participants That Selected ADHD by Vignette Type
Vignette Type,,,
Combined Female,9,11,0.818182
Combined Male,9,14,0.642857
Inattentive Female,5,11,0.454545
Inattentive Male,7,14,0.500000


## 3. Descriptive Table (minimum, maximum, mean, median, SD of responses 0-10 on Q25_1)

In [9]:
df.groupby('Vignette Type').describe()['degree_of_need']

,count,mean,std,min,25%,50%,75%,max
Vignette Type,,,,,,,,
Combined Female,11.0,7.363636,2.540580,4.0,5.00,7.0,10.0,10.0
Combined Male,14.0,6.928571,2.585887,3.0,5.00,7.5,9.0,10.0
Inattentive Female,11.0,6.454545,1.916436,2.0,5.50,7.0,7.5,9.0
Inattentive Male,14.0,6.642857,1.736803,3.0,6.25,7.0,7.0,10.0


## 4. Descriptive Table (sum, mean and standard deviation of score of ADHD knowledge scale correct responses)

In [10]:
q4_cols = ['teacher_gender', 'years_of_teaching', 'School Setting']
for col in q4_cols:
    display(df.groupby(col).describe()['teacher_knowledge'])

,count,mean,std,min,25%,50%,75%,max
teacher_gender,,,,,,,,
female,47.0,19.340426,1.550268,15.0,18.5,20.0,20.0,22.0
male,3.0,17.666667,4.932883,12.0,16.0,20.0,20.5,21.0


,count,mean,std,min,25%,50%,75%,max
years_of_teaching,,,,,,,,
5-10 Years,17.0,19.294118,1.649421,15.0,19.0,20.0,20.0,22.0
<5 Years,8.0,18.875000,1.959410,16.0,17.5,20.0,20.0,21.0
>10 Years,25.0,19.320000,1.994158,12.0,19.0,20.0,20.0,22.0


,count,mean,std,min,25%,50%,75%,max
School Setting,,,,,,,,
Charter,2.0,20.000000,0.000000,20.0,20.0,20.0,20.0,20.0
Other,1.0,20.000000,NaN,20.0,20.0,20.0,20.0,20.0
Private,11.0,19.454545,2.018100,15.0,18.5,20.0,20.5,22.0
Public,36.0,19.111111,1.878872,12.0,18.0,19.5,20.0,22.0


## 5. Descriptive Table (% of each factor endorsed as influencing decision to refer)

In [14]:
factors_map = {1 : {'dimension': 'impact of difficulties on the child', 'count': 0},
                2 : {'dimension': 'Impact of difficulties on the child’s peers', 'count': 0},
                3 : {'dimension': 'Personal views about referrals', 'count': 0},
                4 : {'dimension': 'Parent views about referrals', 'count': 0},
                5 : {'dimension': 'Stigma surrounding referrals', 'count': 0},
                6 : {'dimension': 'Personal experience with specialist services', 'count': 0},
                7 : {'dimension': 'Minimal awareness of what specialist services offer', 'count': 0},
                8 : {'dimension': 'Other', 'count': 0},
              }

for i, row in df.iterrows():
    for resp in str(row['Q26']).split(','):
        factors_map[int(resp)]['count']+=1
        
multiValuePctTotal(df, factors_map, 'Factors Endorsed as Influencing Decision to Refer')

,Percent of Total
Factors Endorsed as Influencing Decision to Refer,
impact of difficulties on the child,0.92
Impact of difficulties on the child’s peers,0.20
Personal views about referrals,0.24
Parent views about referrals,0.20
Stigma surrounding referrals,0.06
Personal experience with specialist services,0.54
Minimal awareness of what specialist services offer,0.04
Other,0.12


# Statistical Analyses

## 1. ANOVAS (outcome variable = Q25_1)

In [15]:
# Q25_1 = degree_of_need
#a. student gender and degree_of_need
gender_model = ols('degree_of_need ~ student_gender', data=df).fit()
anova_gender_table = sm.stats.anova_lm(gender_model, typ=2)
anova_gender_table

,sum_sq,df,F,PR(>F)
student_gender,0.187532,1.0,0.038711,0.844853
Residual,232.532468,48.0,NaN,NaN


In [16]:
# Q25_1 = degree_of_need
#b. adhd subtype and degree_of_need
adhd_subtype_model = ols('degree_of_need ~ adhd_subtype', data=df).fit()
anova_adhd_subtype_table = sm.stats.anova_lm(adhd_subtype_model, typ=2)
anova_adhd_subtype_table

,sum_sq,df,F,PR(>F)
adhd_subtype,3.92,1.0,0.822378,0.369016
Residual,228.80,48.0,NaN,NaN


## 2. Logistic Regression

In [17]:
log_model = logit('student_exhibits_adhd ~ adhd_subtype+years_of_teaching+student_gender+teacher_knowledge', data=df).fit()
log_model.summary()

Optimization terminated successfully.
         Current function value: 0.607406
         Iterations 6


<class 'statsmodels.iolib.summary.Summary'>
"""
                             Logit Regression Results                            
=================================================================================
Dep. Variable:     student_exhibits_adhd   No. Observations:                   50
Model:                             Logit   Df Residuals:                       44
Method:                              MLE   Df Model:                            5
Date:                   Tue, 21 Mar 2023   Pseudo R-squ.:                 0.09748
Time:                           21:18:41   Log-Likelihood:                -30.370
converged:                          True   LL-Null:                       -33.651
Covariance Type:               nonrobust   LLR p-value:                    0.2554
==================================================================================================
                                     coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------------------------
Intercept                         -3.1962      3.700     -0.864      0.388     -10.449       4.056
adhd_subtype[T.inattentive]       -1.1275      0.655     -1.720      0.085      -2.412       0.157
years_of_teaching[T.<5 Years]     -0.3711      0.972     -0.382      0.703      -2.277       1.535
years_of_teaching[T.>10 Years]    -0.8508      0.734     -1.159      0.246      -2.290       0.588
student_gender[T.male]            -0.5176      0.659     -0.785      0.433      -1.810       0.775
teacher_knowledge                  0.2600      0.196      1.325      0.185      -0.125       0.645
==================================================================================================
"""

## 3. Linear Regression

In [18]:
lin_model = ols('degree_of_need ~ adhd_subtype+years_of_teaching+student_gender+teacher_knowledge', data=df).fit()
lin_model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:         degree_of_need   R-squared:                       0.147
Model:                            OLS   Adj. R-squared:                  0.050
Method:                 Least Squares   F-statistic:                     1.518
Date:                Tue, 21 Mar 2023   Prob (F-statistic):              0.204
Time:                        21:18:41   Log-Likelihood:                -105.41
No. Observations:                  50   AIC:                             222.8
Df Residuals:                      44   BIC:                             234.3
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
==================================================================================================
                                     coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------------
Intercept                         10.4151      3.262      3.193      0.003       3.841      16.989
adhd_subtype[T.inattentive]       -0.5350      0.619     -0.864      0.392      -1.782       0.712
years_of_teaching[T.<5 Years]     -2.3036      0.915     -2.518      0.016      -4.147      -0.460
years_of_teaching[T.>10 Years]    -0.7412      0.699     -1.060      0.295      -2.150       0.668
student_gender[T.male]            -0.2141      0.625     -0.342      0.734      -1.474       1.046
teacher_knowledge                 -0.1273      0.166     -0.766      0.448      -0.462       0.208
==============================================================================
Omnibus:                        2.133   Durbin-Watson:                   1.869
Prob(Omnibus):                  0.344   Jarque-Bera (JB):                1.857
Skew:                          -0.466   Prob(JB):                        0.395
Kurtosis:                       2.854   Cond. No.                         211.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

# Chi-Squared Tests

In [20]:
contingency= pd.crosstab(df['teachers_view'], df['Vignette Type'])
contingency

Vignette Type,Combined Female,Combined Male,Inattentive Female,Inattentive Male
teachers_view,,,,
"1,2,3,5,6,7",0,1,0,0
"1,4,7",0,1,0,0
"1,6,7",0,0,1,0
"1,7",0,1,0,0
"2,3,4,5,6,7",0,2,0,1
"2,3,5",1,0,0,0
"2,3,5,6,7",1,0,0,0
"2,3,5,7",1,1,1,1
"2,3,7",0,0,0,1


In [45]:
teachers_view_map

{1: {'dimension': '1', 'count': 4},
 2: {'dimension': '2', 'count': 17},
 3: {'dimension': '3', 'count': 20},
 4: {'dimension': '4', 'count': 8},
 5: {'dimension': 'ADHD', 'count': 30},
 6: {'dimension': '6', 'count': 13},
 7: {'dimension': '7', 'count': 49}}

In [46]:
df_teacher_view_and_vignette = df[['teachers_view','Vignette Type']]
df_teacher_view_and_vignette.loc[:,'teachers_view_arr'] = df_teacher_view_and_vignette.loc[:,'teachers_view'].str.split(',')
df_teacher_view_and_vignette

/var/folders/z_/b2p80qvn353dbq0c1vfq0q600000gq/T/ipykernel_71565/938823564.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_teacher_view_and_vignette.loc[:,'teachers_view_arr'] = df_teacher_view_and_vignette.loc[:,'teachers_view'].str.split(',')


,teachers_view,Vignette Type,teachers_view_arr
0,"2,3,7",Inattentive Male,"[2, 3, 7]"
1,"2,3,4,5,6,7",Inattentive Male,"[2, 3, 4, 5, 6, 7]"
2,"3,5,7",Inattentive Female,"[3, 5, 7]"
3,"2,3,5,7",Combined Female,"[2, 3, 5, 7]"
4,"3,5,7",Combined Female,"[3, 5, 7]"
5,"2,3,5,7",Inattentive Male,"[2, 3, 5, 7]"
6,7,Combined Male,[7]
7,"1,4,7",Combined Male,"[1, 4, 7]"
8,"3,5,7",Inattentive Male,"[3, 5, 7]"
9,"2,3,5,7",Inattentive Female,"[2, 3, 5, 7]"


In [47]:
df_teacher_view_and_vignette = df_teacher_view_and_vignette.explode('teachers_view_arr').reset_index(drop=True)
df_teacher_view_and_vignette

,teachers_view,Vignette Type,teachers_view_arr
0,"2,3,7",Inattentive Male,2
1,"2,3,7",Inattentive Male,3
2,"2,3,7",Inattentive Male,7
3,"2,3,4,5,6,7",Inattentive Male,2
4,"2,3,4,5,6,7",Inattentive Male,3
...,...,...,...
136,"2,3,5",Combined Female,3
137,"2,3,5",Combined Female,5
138,"1,6,7",Inattentive Female,1
139,"1,6,7",Inattentive Female,6


# Updated Data Needs (3/21)

In [38]:
vign_val_counts = dict(df['Vignette Type'].value_counts())

### Helper Functions

In [88]:
def getContingencyPctTable(df, indx, rename_indx,  col_breakout, distinct_possible_answers):
    df_cont = df[[indx,col_breakout]].copy()
    df_cont.loc[:,indx] = df_cont.loc[:,indx].str.split(',')
    df_cont = df_cont.explode(indx).reset_index(drop=True)
    df_cont =  pd.crosstab(df_cont[indx], 
                        df_cont[col_breakout])
    row_list = []
    for i in range(1,distinct_possible_answers+1):
        row = {}
        for col in df_cont:
            row[rename_indx] = i
            row[col] = df_cont.loc[str(i), col]/vign_val_counts[col]
        row_list.append(row)
    df_final = pd.DataFrame(row_list).set_index([rename_indx])
    return df_final

### 1. Descriptives of responses for Q22 ("In your opinion, does the student exhibit the following?" - can select as many as they wanted) Want n (%) for each response choice by vignette group (inattentive boy, combined boy, inattentive girl, combined girl)

In [90]:
getContingencyPctTable(df, 'teachers_view','Teachers View', 'Vignette Type', 7).rename(index={1:'Typical behavior for a child', 
                               2:'Learning Difficulties',
                               3:'Hyperactivity',
                               4:'Emotional difficulties',
                               5:'ADHD',
                               6:'Behavioral difficulties',
                               7:'Attention difficulties'})

,Combined Female,Combined Male,Inattentive Female,Inattentive Male
Teachers View,,,,
Typical behavior for a child,0.000000,0.214286,0.090909,0.000000
Learning Difficulties,0.272727,0.428571,0.272727,0.357143
Hyperactivity,0.545455,0.500000,0.181818,0.357143
Emotional difficulties,0.000000,0.214286,0.272727,0.142857
ADHD,0.818182,0.642857,0.454545,0.500000
Behavioral difficulties,0.272727,0.285714,0.181818,0.285714
Attention difficulties,0.909091,1.000000,1.000000,1.000000


### 2. Descriptives of responses for Q23 ("before considering a referral to specialist services, would you consider any of the following?" - can select as many as they wanted) Want n (%) for each response choice by vignette group

In [92]:
# df['Q23'].value_counts()
getContingencyPctTable(df, 'Q23', 'Before referral, would you consult', 'Vignette Type', 5).rename(
                        index={1:'Parents', 
                               2:'School nurse',
                               3:'Colleague',
                               4:'School psychologist',
                               5:'Not listed (please describe)'}
                    )

,Combined Female,Combined Male,Inattentive Female,Inattentive Male
"Before referral, would you consult",,,,
Parents,0.818182,0.785714,1.000000,0.857143
School nurse,0.181818,0.142857,0.272727,0.071429
Colleague,1.000000,0.785714,0.818182,0.571429
School psychologist,0.909091,0.928571,0.909091,1.000000
Not listed (please describe),0.090909,0.142857,0.181818,0.214286


### 3. Descriptives of responses for Q27 ("considering the vignette, do you believe this student would benefit from any of the following interventions?" - can select as many as they wanted) Want n (%) for each response choice by vignette group

In [93]:
getContingencyPctTable(df, 'Q27', 'Beneficial Interventions', 'Vignette Type', 4).rename(
                        index={1:'Medication', 
                               2:'Additional academic-focused support',
                               3:'Behavioral interventions',
                               4:'Not listed (please describe)'}
                    )

,Combined Female,Combined Male,Inattentive Female,Inattentive Male
Beneficial Interventions,,,,
Medication,0.363636,0.571429,0.363636,0.285714
Additional academic-focused support,0.909091,0.857143,1.000000,0.928571
Behavioral interventions,0.818182,0.714286,0.454545,0.785714
Not listed (please describe),0.181818,0.142857,0.363636,0.142857


### 4. Descriptives of responses on Q25 (degree of need) - see attached screenshot titled degree of need for an example of what I'm looking for, except instead of vignette type on the y axis, we want the following (mean, median, min, max, std:

In [94]:
# % Total Descriptive Statistics
desc_columns=['teacher_gender', 'years_of_teaching', 'School Setting', 'Geographical Area', 'Additional Certification']
for col in desc_columns:
    display(df.groupby(col).describe()['degree_of_need'])

,count,mean,std,min,25%,50%,75%,max
teacher_gender,,,,,,,,
female,47.0,6.872340,2.232342,2.0,5.0,7.0,8.5,10.0
male,3.0,6.333333,1.154701,5.0,6.0,7.0,7.0,7.0


,count,mean,std,min,25%,50%,75%,max
years_of_teaching,,,,,,,,
5-10 Years,17.0,7.588235,2.063335,3.0,7.0,8.0,9.0,10.0
<5 Years,8.0,5.375000,1.767767,4.0,4.0,5.0,5.5,9.0
>10 Years,25.0,6.800000,2.198484,2.0,6.0,7.0,8.0,10.0


,count,mean,std,min,25%,50%,75%,max
School Setting,,,,,,,,
Charter,2.0,7.500000,3.535534,5.0,6.25,7.5,8.75,10.0
Other,1.0,7.000000,NaN,7.0,7.00,7.0,7.00,7.0
Private,11.0,6.545455,2.296242,4.0,4.50,6.0,8.50,10.0
Public,36.0,6.888889,2.174893,2.0,5.75,7.0,8.00,10.0


,count,mean,std,min,25%,50%,75%,max
Geographical Area,,,,,,,,
Suburban,40.0,6.95,2.123736,2.0,5.75,7.0,8.0,10.0
Urban,10.0,6.40,2.458545,4.0,4.25,5.5,8.5,10.0


,count,mean,std,min,25%,50%,75%,max
Additional Certification,,,,,,,,
No,12.0,6.250000,1.959824,3.0,4.75,7.0,7.0,10.0
Yes,38.0,7.026316,2.235909,2.0,5.00,7.0,9.0,10.0


In [101]:
df_classroom = df[['Q15','degree_of_need']]
df_classroom.loc[:,'Type of Classroom'] = df_classroom.loc[:,'Q15'].str.split(',')
df_classroom = df_classroom.explode('Type of Classroom')
df_classroom.groupby('Type of Classroom').describe()['degree_of_need']

/var/folders/z_/b2p80qvn353dbq0c1vfq0q600000gq/T/ipykernel_71565/814006390.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_classroom.loc[:,'Type of Classroom'] = df_classroom.loc[:,'Q15'].str.split(',')


,count,mean,std,min,25%,50%,75%,max
Type of Classroom,,,,,,,,
1,20.0,6.750000,1.773341,3.0,5.75,7.0,7.00,10.0
2,13.0,6.923077,2.139374,3.0,5.00,7.0,9.00,10.0
3,17.0,7.529412,1.907801,4.0,6.00,8.0,9.00,10.0
4,12.0,6.166667,2.587850,2.0,4.00,7.0,7.25,10.0


### 5. Descriptives for Q5 (teacher age) can you give me the min, max, mean and median of teacher age?

In [95]:
df['Q5'].describe()

count    50.00000
mean     37.06000
std      11.66612
min      23.00000
25%      27.00000
50%      33.50000
75%      49.25000
max      60.00000
Name: Q5, dtype: float64